In [0]:
# 因為圖片實在太多，所以要用壓縮檔載入

import zipfile

f = zipfile.ZipFile("./drive/My Drive/AOI/train_images.zip")
f.extractall("./datasets")

In [0]:
# 圖片都混在一起了，分類一下

from glob import glob
import os
import pandas as pd

# 取得檔案路徑
imagePaths = glob('/content/datasets/train_images/*')

# 取得分類答案
imageAns = pd.read_csv("./drive/My Drive/AOI/train.csv")

# 創新資料夾
for class_num in range(6):
  DATASET_PATH = '/content/train_images/'
  class_folders = DATASET_PATH + str(class_num)

  # exist_ok：只有在目录不存在时创建目录，目录已存在时不会抛出异常。
  os.makedirs(class_folders, exist_ok=True)

In [0]:
# 移動檔案

import shutil

for class_num in range(6):
  class00 = imageAns[imageAns["Label"] == class_num]
  class_list = class00["ID"]
  # print(class_list)

  for c in class_list:
    shutil.move('/content/datasets/train_images/' + str(c), 
                '/content/train_images/' + str(class_num) + "/" + str(c))

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Model
import tensorflow.keras.layers as L

cnn = VGG16(input_shape = (256, 256, 3),
            weights = 'imagenet',
            include_top = False)

for layer in cnn.layers:
    layer.trainable = True

x = cnn.output
x = L.BatchNormalization()(x)
x = L.Activation('relu')(x)
x = L.GlobalAveragePooling2D()(x)
out = L.Dense(6, activation='softmax')(x)

model = Model(inputs=cnn.input, outputs=out)

In [0]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

# 資料放大器
# rescale != VGG16的格式，下次再改。

Generator = ImageDataGenerator(rescale= 1. / 255,       # 像素值縮放
                               rotation_range=40,       # 任意旋轉 ±40 度
                               width_shift_range=0.2,   # 寬度偏移 ±20%
                               height_shift_range=0.2,  # 高度偏移 ±20%
                               shear_range=0.2,         # 錯位 ±20%
                               zoom_range=0.2,          # 縮放 ±20%
                               channel_shift_range=10,  # 通道 偏移 ±10 像素
                               horizontal_flip=True,    # 水平翻轉
                               fill_mode='nearest',     # 填滿模式
                               dtype='float32')         # 設定生成圖片格式 float32

In [7]:
# flow_from_directory = 以文件夹路径为参数，生成经过数据提升/归一化后的数据，在一个无限循环中无限产生batch的数据

DATASET_PATH = '/content/train_images/'

Generator_flow = Generator.flow_from_directory(DATASET_PATH,
                                               target_size= (256, 256),
                                               batch_size= 20,
                                               class_mode= 'categorical')

Found 2528 images belonging to 6 classes.


In [8]:
history = model.fit(Generator_flow,
                    epochs= 10, 
                    verbose= 1)

Epoch 1/10
127/127 [==============================] - 49s 388ms/step - loss: 1.3705 - categorical_accuracy: 0.5277
Epoch 2/10
127/127 [==============================] - 49s 386ms/step - loss: 1.2316 - categorical_accuracy: 0.5518
Epoch 3/10
127/127 [==============================] - 48s 382ms/step - loss: 1.2033 - categorical_accuracy: 0.5281
Epoch 4/10
127/127 [==============================] - 48s 381ms/step - loss: 1.1678 - categorical_accuracy: 0.5562
Epoch 5/10
127/127 [==============================] - 48s 379ms/step - loss: 1.1657 - categorical_accuracy: 0.5403
Epoch 6/10
127/127 [==============================] - 48s 378ms/step - loss: 1.1412 - categorical_accuracy: 0.5506
Epoch 7/10
127/127 [==============================] - 48s 377ms/step - loss: 1.1574 - categorical_accuracy: 0.5407
Epoch 8/10
127/127 [==============================] - 48s 379ms/step - loss: 1.1490 - categorical_accuracy: 0.5427
Epoch 9/10
127/127 [==============================] - 48s 379ms/step - loss: 1.1

In [9]:
# 看準確率

loss, acc = model.evaluate(Generator_flow)
print('準確率:', acc)

127/127 [==============================] - 41s 325ms/step - loss: 1.4285 - categorical_accuracy: 0.4324
準確率: 0.4323576092720032


In [0]:
# 載入 真．驗證資料

import zipfile

f = zipfile.ZipFile("./drive/My Drive/AOI/test_images.zip")
f.extractall("./datasets_test/")

In [11]:
# 寫 Test 的 Generator_flow
# 其實只是資料夾的路徑不同而已

Generator_flow_Test = Generator.flow_from_directory('/content/datasets_test',
                                                    target_size= (256, 256),
                                                    batch_size= 20,
                                                    class_mode= 'categorical')

Found 10142 images belonging to 1 classes.


In [12]:
# 吃我的預測拉

pred = model.predict_generator(Generator_flow_Test, verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
508/508 [==============================] - 168s 330ms/step


In [13]:
# 預言出來了

import numpy as np

predicted_class_indices = np.argmax(pred, axis=1)
predicted_class_indices

array([0, 0, 0, ..., 0, 5, 5])

In [0]:
pre_df = pd.DataFrame({"機器預測":predicted_class_indices})
# pre_df

In [15]:
import pandas as pd

df2 = pd.read_csv("./drive/My Drive/AOI/test.csv")
df2 = df2.drop(["Label"], axis=1)

result = pd.concat([df2, pre_df], axis=1) 
result

,ID,機器預測
0,test_00000.png,0
1,test_00001.png,0
2,test_00002.png,0
3,test_00003.png,0
4,test_00004.png,5
...,...,...
10137,test_10137.png,0
10138,test_10138.png,5
10139,test_10139.png,0
10140,test_10140.png,5
